In [23]:
import pandas as pd
import utils 
import plotly.graph_objects as go 

In [24]:
pair = "EUR_USD"
granularity = 'H1'
ma_list = [8,16,32,64,128,256]

In [25]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity) )

In [26]:
non_cols =['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

set mod_cols = to a list and get us each value in the  dataframe column if that 'x' is not in the list, then create a new list 
this happens often that we need to convert strings to numbers or any other data type 

In [27]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [34]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2023-12-20T23:00:00.000000000Z,1.09442,1.09464,1.09429,1.09432,1.094786,1.095254,1.096278,1.094645,1.093680,1.085695
256,2023-12-21T00:00:00.000000000Z,1.09434,1.09502,1.09410,1.09499,1.094705,1.095185,1.096193,1.094695,1.093803,1.085759
257,2023-12-21T01:00:00.000000000Z,1.09500,1.09534,1.09483,1.09524,1.094635,1.095110,1.096123,1.094746,1.093934,1.085822
258,2023-12-21T02:00:00.000000000Z,1.09522,1.09545,1.09452,1.09495,1.094505,1.095091,1.096043,1.094806,1.094062,1.085884
259,2023-12-21T03:00:00.000000000Z,1.09497,1.09500,1.09465,1.09492,1.094536,1.095164,1.095959,1.094863,1.094110,1.085950


In [29]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [30]:
df_ma.dropna(inplace=True)

In [35]:
df_plot = df_ma.iloc[-200:].copy()

In [32]:
df_plot.shape

(100, 11)

In [36]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()